## Import packages and modules

In [1]:
from models.tda_models import TDA_PI34_Model, TDA_PI42_Model, VECTOR_STITCHING_PI_Model_34, VECTOR_STITCHING_PI_Model_42
from models.raw_models import Raw_Model, Dummy_Model
from tda_pipelines import *
import tensorflow as tf


## Load TDA and Vector-stitching pipelines 

In [2]:
# load pipelines 

tda_pipeline_34 = TDA_PI34_Pipeline()
tda_pipeline_42 = TDA_PI42_Pipeline()
vector_stitching_pipeline_34, tda_union_34 = VECTOR_STITCHING_PI_Pipeline_34()
vector_stitching_pipeline_42, tda_union_42 = VECTOR_STITCHING_PI_Pipeline_42()


## Transform raw data to persistance images and stitched RAW-PI images

In [3]:
from data_preprocessing import process
(X, y), _ = tf.keras.datasets.mnist.load_data()

data = process(data=(X, y), training_size=100, testing_size=10, dist_ratio=10)

c:\Users\jawor\Desktop\Projekt_licencjacki\data_preprocessing.py:47: FutureWarning: `seed` is a deprecated argument name for `random_noise`. It will be removed in version 0.23. Please use `rng` instead.
  X_train_noisy = random_noise(X_train, mode="s&p",amount=0.05, seed=666)
c:\Users\jawor\Desktop\Projekt_licencjacki\data_preprocessing.py:48: FutureWarning: `seed` is a deprecated argument name for `random_noise`. It will be removed in version 0.23. Please use `rng` instead.
  X_test_noisy = random_noise(X_test, mode="s&p",amount=0.05, seed=666)


1/8
2/8
3/8
4/8
5/8
6/8
7/8
8/8


# Initialize models

In [4]:
# models

raw_model = Raw_Model() # cnn models working on raw images
dummy_model = Dummy_Model() # fully dense model working on raw images
tda_model_34 = TDA_PI34_Model() # cnn model working on persistance images
tda_model_42 = TDA_PI42_Model() # cnn model working on persistance images
vector_stitching_model_34 = VECTOR_STITCHING_PI_Model_34() # cnn model working on stitched raw and PI images
vector_stitching_model_42 = VECTOR_STITCHING_PI_Model_42() # cnn model working on stitched raw and PI images


# Fitting and validating models


In [5]:
tf.keras.backend.clear_session()

In [6]:
#TDA model

tda_model_34.model.fit(data["X_tr_tda_34"], data["y_tr_tda_34"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_tda_good_34"], data["y_test"]))

Epoch 1/20


100/100 [==============================] - 5s 14ms/step - loss: 2.9130 - accuracy: 0.1400 - val_loss: 2.3608 - val_accuracy: 0.1000
Epoch 2/20
100/100 [==============================] - 1s 12ms/step - loss: 2.0023 - accuracy: 0.3700 - val_loss: 2.5890 - val_accuracy: 0.2000
Epoch 3/20
100/100 [==============================] - 1s 13ms/step - loss: 1.9220 - accuracy: 0.5100 - val_loss: 3.5322 - val_accuracy: 0.2000
Epoch 4/20
100/100 [==============================] - 1s 15ms/step - loss: 1.3377 - accuracy: 0.5800 - val_loss: 1.9630 - val_accuracy: 0.3000
Epoch 5/20
100/100 [==============================] - 1s 14ms/step - loss: 0.8506 - accuracy: 0.7600 - val_loss: 1.6495 - val_accuracy: 0.6000
Epoch 6/20
100/100 [==============================] - 1s 11ms/step - loss: 0.7769 - accuracy: 0.7600 - val_loss: 4.4403 - val_accuracy: 0.0000e+00
Epoch 7/20
100/100 [==============================] - 2s 17ms/step - loss: 1.3400 - accuracy: 0.6500 - val_loss: 1.8151 - val_accuracy: 

In [10]:
tf.keras.backend.clear_session()

In [11]:
# RAW model
raw_model.model.fit(data["X_tr"], data["y_tr"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_random"], data["y_test"]))

Epoch 1/20
100/100 [==============================] - 3s 16ms/step - loss: 6.2304 - accuracy: 0.5300 - val_loss: 2.4220 - val_accuracy: 0.1000
Epoch 2/20
100/100 [==============================] - 1s 13ms/step - loss: 1.3970 - accuracy: 0.4900 - val_loss: 2.3114 - val_accuracy: 0.1000
Epoch 3/20
100/100 [==============================] - 1s 12ms/step - loss: 0.6802 - accuracy: 0.7900 - val_loss: 2.3154 - val_accuracy: 0.1000
Epoch 4/20
100/100 [==============================] - 1s 13ms/step - loss: 0.3357 - accuracy: 0.9000 - val_loss: 2.4247 - val_accuracy: 0.1000
Epoch 5/20
100/100 [==============================] - 1s 14ms/step - loss: 0.2902 - accuracy: 0.9200 - val_loss: 2.2875 - val_accuracy: 0.1000
Epoch 6/20
100/100 [==============================] - 1s 14ms/step - loss: 0.2352 - accuracy: 0.9200 - val_loss: 2.3583 - val_accuracy: 0.1000
Epoch 7/20
100/100 [==============================] - 1s 14ms/step - loss: 0.4296 - accuracy: 0.8700 - val_loss: 2.2480 - val_accuracy: 0.1000

In [ ]:
i=0
combined_score = 0
for row_tda, row_raw in zip(predictions_tda, predictions_raw):
    # multiply rows elementwise 
    combined = row_tda * row_raw
    if np.argmax(combined) == y_test[i]:
        combined_score += 1
    #print("tda: ",np.argmax(row_tda),"    raw: ", np.argmax(row_raw), "    combined: ", np.argmax(combined), "    actual: ", y_test[i]) 
    i+=1

print(f"Combined Accuracy: {combined_score/len(predictions_tda)}")

tda:  3     raw:  5     combined:  3     actual:  4
tda:  5     raw:  5     combined:  5     actual:  5
tda:  4     raw:  5     combined:  4     actual:  6
tda:  3     raw:  1     combined:  3     actual:  1
tda:  1     raw:  5     combined:  1     actual:  2
tda:  1     raw:  1     combined:  1     actual:  4
tda:  1     raw:  1     combined:  1     actual:  8
tda:  0     raw:  5     combined:  0     actual:  0
tda:  0     raw:  5     combined:  0     actual:  8
tda:  1     raw:  1     combined:  1     actual:  7
tda:  9     raw:  1     combined:  9     actual:  7
tda:  5     raw:  1     combined:  5     actual:  4
tda:  1     raw:  5     combined:  1     actual:  4
tda:  1     raw:  1     combined:  1     actual:  2
tda:  1     raw:  1     combined:  1     actual:  3
tda:  6     raw:  5     combined:  6     actual:  6
tda:  1     raw:  5     combined:  1     actual:  6
tda:  1     raw:  5     combined:  1     actual:  7
tda:  8     raw:  5     combined:  8     actual:  2
tda:  8     

In [14]:
tf.keras.backend.clear_session()

In [15]:
#Vector stitching model

vector_stitching_model_34.model.fit(data["X_tr_vector_stitching_34"], data["y_tr_vector_stitching_34"], epochs=20, batch_size=1, validation_data=(data["X_test_noisy_vector_stitching_good_34"], data["y_test"]))

Epoch 1/20
100/100 [==============================] - 3s 18ms/step - loss: 1.2522 - accuracy: 0.7200 - val_loss: 1.3980 - val_accuracy: 0.6000
Epoch 2/20
100/100 [==============================] - 1s 12ms/step - loss: 1.6673 - accuracy: 0.8100 - val_loss: 2.6124 - val_accuracy: 0.3000
Epoch 3/20
100/100 [==============================] - 2s 24ms/step - loss: 1.1328 - accuracy: 0.7600 - val_loss: 1.3812 - val_accuracy: 0.5000
Epoch 4/20
100/100 [==============================] - 2s 21ms/step - loss: 0.3687 - accuracy: 0.8700 - val_loss: 1.0766 - val_accuracy: 0.8000
Epoch 5/20
100/100 [==============================] - 2s 18ms/step - loss: 0.1965 - accuracy: 0.9500 - val_loss: 1.7564 - val_accuracy: 0.8000
Epoch 6/20
100/100 [==============================] - 1s 11ms/step - loss: 0.5280 - accuracy: 0.9400 - val_loss: 1.5996 - val_accuracy: 0.7000
Epoch 7/20
100/100 [==============================] - 1s 11ms/step - loss: 0.7553 - accuracy: 0.8800 - val_loss: 0.8021 - val_accuracy: 0.8000